## Imports and generic functions

In [1]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn_extra.cluster import KMedoids
import copy
from constants import *
from evaluation import *
from real_time_adj_original import *

In [2]:
(prices_B,prices_S,prices_F,prices_forecast,features,features_red,realized,price_H,max_elec,nominal_wind,penalty,min_production, forecast_production) = import_consts()

month = 30*24
year = 8760
idx_start = len(prices_F) - year
idx_end = len(prices_F)
print(len(realized))

## Testing the models

#### Deterministic and hindsight (and example HAPD-AF-12)

In [3]:
#-----------Deterministic
df_det = pd.read_csv("./results/2020/deterministic.csv")

result_det = test_fixed(df_det.loc[:, 'forward bid'], df_det.loc[:, 'hydrogen production'], idx_start, idx_end)
print(f"Deterministic model all bids acccepted : {np.sum(result_det['obj'])}")

result_det_strict = test_det(df_det.loc[:, 'forward bid'], df_det.loc[:, 'hydrogen production'], idx_start, idx_end)
print(f"Deterministic model : {np.sum(result_det_strict['obj'])}")


#-----------hindsight
df_hs = pd.read_csv("./results/2020/hindsight.csv")

result_hs = test_fixed(df_hs.loc[:, 'forward bid'], df_hs.loc[:, 'hydrogen production'], idx_start, idx_end)
print(f"Hindsight model: {np.sum(result_hs['obj'])}")



#-----------HAPD-AF-12
df_hapd_af_12 = pd.read_csv("./results/2020/HAPD-AF-12.csv")

result_hapd_af_12 = test_price_domain(df_hapd_af_12, idx_start, idx_end)
print(f"HAPD-AF-12: {np.sum(result_hapd_af_12['obj'])}")


#### General models

##### General

In [ ]:

#--------------------------GENERAL-----------------

def test_general(model_name, fm=False, reduced_features=False):
    results = []
    print("\nTesting model ", model_name)
    for i in range(12):
        df = pd.read_csv(f"./results/2020/comparing_architecture/{model_name}_mo{i+1}.csv")
        df_f = df[[f"qF{j+1}" for j in range(int(len(df.columns)/2))]]
        df_h = df[[f"qH{j+1}" for j in range(int(len(df.columns)/2))]]
        tester_results = test_initial_plan(df_f, df_h, idx_start, idx_end, True, fm, reduced_features)
        results.append(tester_results)
        print(f"{i+1} months training: {np.sum(tester_results['obj'])}")
    return results

# # GENERAL For all features 
results_g_all = test_general("GENERAL")

# GENERAL Only forecast and price features 
results_g_rf = test_general("GENERAL_rf", False, True)

# GENERAL Forecast 
results_g_fm = test_general("GENERAL_fm", True)


##### General oracle

In [ ]:
#--------------------------GENERAL-----------------

def test_general_oracle(model_name, fm=False, reduced_features=False):
    results = []
    print("\nTesting model ", model_name)
    df = pd.read_csv(f"./results/2020/comparing_architecture/{model_name}.csv")
    df_f = df[[f"qF{j+1}" for j in range(int(len(df.columns)/2))]]
    df_h = df[[f"qH{j+1}" for j in range(int(len(df.columns)/2))]]
    tester_results = test_initial_plan(df_f, df_h, idx_start, idx_end, True, fm, reduced_features)
    results.append(tester_results)
    print(f"Results: {np.sum(tester_results['obj'])}")
    return results

# GENERAL For all features - best was 
results_g_all_oracle = test_general_oracle("GENERAL_ORACLE")

# GENERAL Only forecast and price features - best was 
results_g_rf_oracle = test_general_oracle("GENERAL_rf_ORACLE", False, True)

# GENERAL Forecast model - best was 
results_g_fm_oracle = test_general_oracle("GENERAL_fm_ORACLE", True)


##### General PD

In [ ]:

#--------------------------general_PRICEDOMAIN-----------------

def test_general_pd(model_name, fm=False, reduced_features=False):
    results = []
    print("\nTesting model ", model_name)
    for i in range(12):
        df = pd.read_csv(f"./results/2020/pricedomains/{model_name}_mo{i+1}.csv")
        tester_results = test_price_domain(df, idx_start, idx_end, True, fm, reduced_features)
        results.append(tester_results)
        print(f"{i+1} months training: {np.sum(tester_results['obj'])}")
    return results

# general_PRICEDOMAIN For all features
results_g_all_PD = test_general_pd("general_PRICEDOMAIN")

# general_PRICEDOMAIN Only forecast and price features
results_g_rf_PD = test_general_pd("general_rf_PRICEDOMAIN", False, True)

# general_PRICEDOMAIN Forecast model 
results_g_fm_PD = test_general_pd("general_fm_PRICEDOMAIN", True)



##### General pd Retraining

In [ ]:
#--------------------------general_pricedomain_RETRAINING-----------------

def test_general_pd_retraining(init_filename, model_name, fm=False, reduced_features=False):
    results = []
    print("\nTesting model ", model_name)
    for i in range(12):
        cqs_filename = (f"./results/2020/changing_qs_general/{i+1}_month_training/{model_name}")
        tester_results = test_price_domain_changing_qs(init_filename, cqs_filename, idx_start, idx_end, True, fm, reduced_features)
        results.append(tester_results)
        print(f"{i+1} months training: {np.sum(tester_results['obj'])}")
    return results

# general_pricedomain_RETRAINING For all features
s_all_pd_init = f"./results/2020/pricedomains/general_PRICEDOMAIN_mo1.csv"
results_g_all_PD_retrained = test_general_pd_retraining(s_all_pd_init, "general_pricedomain_RETRAINING")

# general_pricedomain_RETRAINING Only forecast and price features
s_rf_pd_init = f"./results/2020/pricedomains/general_rf_PRICEDOMAIN_mo1.csv"
results_g_rf_PD_retrained = test_general_pd_retraining(s_rf_pd_init, "general_rf_pricedomain_RETRAINING", False, True)

# general_pricedomain_RETRAINING For all features
s_fm_pd_init = f"./results/2020/pricedomains/general_fm_PRICEDOMAIN_mo1.csv"
results_g_fm_PD_retrained = test_general_pd_retraining(s_fm_pd_init, "general_forecasted_fm_pricedomain_RETRAINING", True)


#### Hourly models

##### Hourly

In [ ]:

#--------------------------HOURLY-----------------

def test_hourly(model_name, fm=False, reduced_features=False):
    results = []
    print("\nTesting model ", model_name)
    for i in range(12):
        df = pd.read_csv(f"./results/2020/comparing_architecture/{model_name}_mo{i+1}.csv")
        df_f = df[[f"qF{j+1}" for j in range(int(len(df.columns)/2))]]
        df_h = df[[f"qH{j+1}" for j in range(int(len(df.columns)/2))]]
        tester_results = test_initial_plan(df_f, df_h, idx_start, idx_end, False, fm, reduced_features)
        results.append(tester_results)
        print(f"{i+1} months training: {np.sum(tester_results['obj'])}")
    return results

# HOURLY For all features 
results_h_all = test_hourly("HOURLY")

# HOURLY Only forecast and price features 
results_h_rf = test_hourly("HOURLY_rf", False, True)

# HOURLY Forecast model 
results_h_fm = test_hourly("HOURLY_fm", True)


##### Hourly oracle

In [ ]:

#--------------------------HOURLY oracle-----------------

def test_hourly_oracle(model_name, fm=False, reduced_features=False):
    results = []
    print("\nTesting model ", model_name)
    df = pd.read_csv(f"./results/2020/comparing_architecture/{model_name}.csv")
    df_f = df[[f"qF{j+1}" for j in range(int(len(df.columns)/2))]]
    df_h = df[[f"qH{j+1}" for j in range(int(len(df.columns)/2))]]
    tester_results = test_initial_plan(df_f, df_h, idx_start, idx_end, False, fm, reduced_features)
    results.append(tester_results)
    print(f"result: {np.sum(tester_results['obj'])}")
    return results

# HOURLY For all features 
results_h_all_oracle = test_hourly_oracle("HOURLY_ORACLE")

# HOURLY Only forecast and price features 
results_h_rf_oracle = test_hourly_oracle("HOURLY_rf_ORACLE", False, True)

# HOURLY Forecast model 
results_h_fm_oracle = test_hourly_oracle("HOURLY_fm_ORACLE", True)


##### Hourly PD

In [ ]:

#--------------------------hourly_PRICEDOMAIN-----------------

def test_hourly_pd(model_name, fm=False, reduced_features=False):
    results = []
    print("\nTesting model ", model_name)
    for i in range(12):
        df = pd.read_csv(f"./results/2020/pricedomains/{model_name}_mo{i+1}.csv")
        tester_results = test_price_domain(df, idx_start, idx_end, False, fm, reduced_features)
        results.append(tester_results)
        print(f"{i+1} months training: {np.sum(tester_results['obj'])}")
    return results

# hourly_PRICEDOMAIN For all features 
results_h_all_PD = test_hourly_pd("hourly_PRICEDOMAIN")

# hourly_PRICEDOMAIN Only forecast and price features 
results_h_rf_PD = test_hourly_pd("hourly_rf_PRICEDOMAIN", False, True)

# hourly_PRICEDOMAIN Forecast model 
results_h_fm_PD = test_hourly_pd("hourly_fm_PRICEDOMAIN", True)


##### Hourly retraining

In [ ]:

def test_hourly_retrained(init_filename, model_name, fm=False, reduced_features=False, weekly=False):
    results = []
    print("\nTesting monthly model ", model_name)
    for i in range(12):
        cqs_filename = (f"./results/2020/changing_qs_hourly/{i+1}_month_training/{model_name}")
        tester_results = test_initial_plan_changing_qs(init_filename, cqs_filename, idx_start, idx_end, False, fm, reduced_features, weekly)
        results.append(tester_results)
        print(f"{i+1} months training: {np.sum(tester_results['obj'])}")
    return results

m_rf_init = f"./results/2020/comparing_architecture/hourly_rf_mo3.csv"
results_h_rf_retrained = test_hourly_retrained(m_rf_init, "hourly_rf_RETRAINING", False, True)

m_fm_init = f"./results/2020/comparing_architecture/HOURLY_fm_mo3.csv"
results_h_fm_retrained = test_hourly_retrained(m_fm_init, "hourly_fm_RETRAINING", True)


##### Hourly pd Oracle

In [ ]:

#--------------------------hourly_pricedomain_ORACLE-----------------

def test_hourly_pd_oracle(model_name, fm=False, reduced_features=False):
    results = []
    print("\nTesting model ", model_name)
    df = pd.read_csv(f"./results/2020/pricedomains/{model_name}.csv")
    tester_results = test_price_domain(df, idx_start, idx_end, False, fm, reduced_features)
    results.append(tester_results)
    print(f"Result: {np.sum(tester_results['obj'])}")
    return results

# hourly_PRICEDOMAIN For all features 
results_h_all_pd_ORACLE = test_hourly_pd_oracle("hourly_pricedomain_ORACLE")

# hourly_PRICEDOMAIN Only forecast and price features 
results_h_rf_pd_ORACLE = test_hourly_pd_oracle("hourly_rf_pricedomain_ORACLE", False, True)

# hourly_PRICEDOMAIN Forecast model 
results_h_fm_pd_ORACLE = test_hourly_pd_oracle("hourly_fm_pricedomain_ORACLE", True)


##### Hourly pd Retraining

In [ ]:
#--------------------------hourly_pricedomain_RETRAINING-----------------

def test_hourly_pd_retraining(init_filename, model_name, fm=False, reduced_features=False):
    results = []
    print("\nTesting monthly model ", model_name)
    for i in range(12):
        cqs_filename = (f"./results/2020/changing_qs_hourly/{i+1}_month_training/{model_name}")
        tester_results = test_price_domain_changing_qs(init_filename, cqs_filename, idx_start, idx_end, False, fm, reduced_features)
        results.append(tester_results)
        print(f"{i+1} months training: {np.sum(tester_results['obj'])}")
    return results


# hourly_pricedomain_RETRAINING For all features
m_all_pd_init = f"./results/2020/pricedomains/hourly_PRICEDOMAIN_mo1.csv"
results_h_all_PD_retrained = test_hourly_pd_retraining(m_all_pd_init, "hourly_pricedomain_RETRAINING")

# hourly_pricedomain_RETRAINING Only forecast and price features
m_rf_pd_init = f"./results/2020/pricedomains/hourly_rf_PRICEDOMAIN_mo3.csv"
results_h_rf_PD_retrained = test_hourly_pd_retraining(m_rf_pd_init, "hourly_rf_pricedomain_RETRAINING", False, True)

# # hourly_pricedomain_RETRAINING Forecast model
m_fm_pd_init = f"./results/2020/pricedomains/hourly_fm_PRICEDOMAIN_mo3.csv"
results_h_fm_PD_retrained = test_hourly_pd_retraining(m_fm_pd_init, "hourly_fm_pricedomain_RETRAINING", True)



In [ ]:
def printstuff(r1, r2, trange, maxcount=10):
    # To perform detailed investigations of model outputs
    counter = 0
    for i in trange:
        if counter > maxcount:
            break
        if round(r2['obj'][i], 0) < round(r1['obj'][i], 0):
            counter += 1
            t = i+idx_start
            print(f"\ni={i}, fp={prices_F[t]}, sp={prices_S[t]}, bp={prices_B[t]}, realized={realized[t]}, opt_h={get_hydro_opt(realized[t]-r1['forward_bid'][i], price_H, prices_S[t], prices_B[t], max_elec)}")
            print(f"ori: {r1['obj'][i]:.2f}, fb={r1['forward_bid'][i]:.2f}, hprod={r1['h_prod'][i]:.2f}, dw={r1['dw'][i]:.2f}, up={r1['up'][i]:.2f}, missing={r1['missing_production'][i]:.2f}")
            print(f"should-be-better: {r2['obj'][i]:.2f}, fb={r2['forward_bid'][i]:.2f}, hprod={r2['h_prod'][i]:.2f}, dw={r2['dw'][i]:.2f}, up={r2['up'][i]:.2f}, missing={r2['missing_production'][i]:.2f}")

# Test on when model HAPD-FM-2 was different from optimal as example:
# printstuff(results_h_fm_PD_retrained[1], result_hs, range(idx_end-idx_start))


#### Adjustment

In [ ]:
def adjust(res):
    # Apply all 3 kinds of adjustment to a model
    res_up = apply_upwards_adj(res, idx_start, idx_end)
    res_up_dw = apply_up_and_dw_adj(res, idx_start, idx_end)
    res_pp = apply_risky_policy(res, idx_start, idx_end)
    return res_up, res_up_dw, res_pp

##### Deterministic adjustment

In [ ]:
det_up = apply_upwards_adj(result_det, idx_start, idx_end)
det_up_dw = apply_up_and_dw_adj(result_det, idx_start, idx_end)
det_pp = apply_risky_policy(result_det, idx_start, idx_end)
print("Original: ", np.sum(result_det['obj']))
print("Upwards adj: ", np.sum(det_up['obj']))
print("Up + dw adj: ", np.sum(det_up_dw['obj']))
print("Postponing adj: ", np.sum(det_pp['obj']))

det_strict_up = apply_upwards_adj(result_det_strict, idx_start, idx_end)
det_strict_up_dw = apply_up_and_dw_adj(result_det_strict, idx_start, idx_end)
det_strict_pp = apply_risky_policy(result_det_strict, idx_start, idx_end)
print("Original: ", np.sum(result_det_strict['obj']))
print("Upwards adj: ", np.sum(det_strict_up['obj']))
print("Up + dw adj: ", np.sum(det_strict_up_dw['obj']))
print("Postponing adj: ", np.sum(det_strict_pp['obj']))

##### General architecture adjustment (GAPD-FM-3)

In [ ]:
other_up = apply_upwards_adj(results_g_fm_PD[2], idx_start, idx_end)
other_up_dw = apply_up_and_dw_adj(results_g_fm_PD[2], idx_start, idx_end)
other_pp = apply_risky_policy(results_g_fm_PD[2], idx_start, idx_end)
print("Original: ", np.sum(results_g_fm_PD[2]['obj']))
print("Upwards adj: ", np.sum(other_up['obj']))
print("Up + dw adj: ", np.sum(other_up_dw['obj']))
print("Postponing adj: ", np.sum(other_pp['obj']))

##### Hourly architecture adjustment (HAPD-FM-3)

In [ ]:
best_up = apply_upwards_adj(results_h_fm_PD_retrained[2], idx_start, idx_end)
best_up_dw = apply_up_and_dw_adj(results_h_fm_PD_retrained[2], idx_start, idx_end)
best_pp = apply_risky_policy(results_h_fm_PD_retrained[2], idx_start, idx_end)
print("Original: ", np.sum(results_h_fm_PD_retrained[2]['obj']))
print("Upwards adj: ", np.sum(best_up['obj']))
print("Up + dw adj: ", np.sum(best_up_dw['obj']))
print("Postponing adj: ", np.sum(best_pp['obj']))

##### Optimal adjustment for HAPD-AF-12

In [ ]:
#-----------HAPD-AF-12
df_hs_adj = pd.read_csv("./results/2020/pricedomains/hourly_PRICEDOMAIN_mo12_opt_adj.csv")

result_hs_adj = test_fixed(df_hs_adj.loc[:, 'forward bid'], df_hs_adj.loc[:, 'hydrogen production'], idx_start, idx_end)
print(f"Optimal adjustment model: {np.sum(result_hs_adj['obj'])}")



##### Deterministic investigation

In [ ]:
counter = 0
zero_fb = 0
zero_fb_sum = 0
nonzero_fb = 0
nonzero_fb_sum = 0
n_opt_max = 0
n_opt_min_dev = 0
n_opt_max_sum = 0
n_opt_min_dev_sum = 0
for i, t in enumerate(range(idx_start, idx_end)):
    # if counter > 15:
    #     break
    if (round(det_up['obj'][i], 0) > round(result_det_strict['obj'][i], 0)):
        opt = get_hydro_opt(realized[t] - result_det_strict['forward_bid'][i], price_H, prices_S[t], prices_B[t], max_elec)
        if (opt == max_elec):
            n_opt_max += 1
        else:
            n_opt_min_dev += 1

        if (round(result_det_strict['forward_bid'][i], 0) == 0):
            zero_fb += 1
            zero_fb_sum += det_up['h_prod'][i] - result_det_strict['h_prod'][i]
            if (opt == max_elec):
                n_opt_max_sum += zero_fb_sum
            else:
                n_opt_min_dev_sum += zero_fb_sum
        else:
            nonzero_fb += 1
            nonzero_fb_sum += det_up['h_prod'][i] - result_det_strict['h_prod'][i]
            if (opt == max_elec):
                n_opt_max_sum += nonzero_fb_sum
            else:
                n_opt_min_dev_sum += nonzero_fb_sum

        # print(f"fb={result_det_strict['forward_bid'][i]}, price FC: {prices_forecast[t]}, fp={prices_F[t]}, deficit_settle price: {prices_B[t]}")


non_strict_res = 0
strict_res = 0
non_strict_res_con = 0
strict_res_con = 0
for i, t in enumerate(range(idx_start, idx_end)):
    if (round(result_det_strict['forward_bid'][i], 0) == 0):
        if (result_det['forward_bid'][i] > 0):
            non_strict_res += prices_F[t] * result_det['forward_bid'][i]
            strict_res += prices_S[t] * result_det['forward_bid'][i]
        else:
            non_strict_res_con += prices_F[t] * result_det['forward_bid'][i]
            strict_res_con += prices_B[t] * result_det['forward_bid'][i]


print(zero_fb)
print(zero_fb_sum)
print("non")
print(nonzero_fb)
print(nonzero_fb_sum)
print(f"max #:", n_opt_max)
print(f"min dev #: ", n_opt_min_dev)
print(f"max sum:", n_opt_max_sum)
print(f"min dev sum: ", n_opt_min_dev_sum)
print("Missed from bad forecasts prod: ", non_strict_res - strict_res)
print("Missed from bad forecasts con: ", non_strict_res_con - strict_res_con)

#### Plots

In [ ]:
#-------------------OVERVIEW PLOT
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.transforms as mtrans
import matplotlib.markers as markers


#------------------------------------------------------------------------
#-----------------------------------------GA & HA-------------------------
#------------------------------------------------------------------------

def plot_ga_and_ha(save=False):

    results_to_plot = [
        [results_g_all, results_g_rf, results_g_fm],
        [results_h_all, results_h_rf, results_h_fm]
    ]

    plot_titles = [
        "GA",
        "HA"
    ]

    fig, axs = plt.subplots(1,2, figsize=(10,5))
    xs = range(1,13)
    optimals = np.array([np.sum(result_hs['obj'])/(1e6) for _ in xs])
    deterministics = np.array([np.sum(result_det_strict['obj'])/(1e6) for _ in xs])
    for i, result_list in enumerate(results_to_plot):
        axs[i].plot(xs, get_scaled_objs(result_list[0]), 'o', color=RED, label="AF")
        axs[i].plot(xs, get_scaled_objs(result_list[2]), '*', color=BLUE, label="FM")
        axs[i].plot(xs, get_scaled_objs(result_list[1]), '+', color=GREEN, alpha=0.8, label="RF")
        axs[i].plot(xs, optimals, '--', color=CBLUE, label="Hindsight")
        # axs[idx1, idx2].plot(xs, oracles[i]-0.03, '--', color="grey", label="Oracle")
        axs[i].plot(xs, deterministics, '--', color=CGREEN, label="Deterministic")
        axs[i].set_title(plot_titles[i], fontweight="bold")
        axs[i].set_xlabel("Months of training")
        axs[i].set_ylabel("Total revenue for test period [M€]")
        axs[i].set_ylim(0.6, 2)
        axs[i].set_xticks(xs)
        axs[i].legend(loc="lower center")


    fig.tight_layout()
    
    # handles = []


    # af_marker = Line2D([0], [0], label='AF', color=RED, marker='o', linestyle='None')
    # fm_marker = Line2D([0], [0], label='FM', color=BLUE, marker='*', linestyle='None')
    # fp_marker = Line2D([0], [0], label='FP', color=GREEN, marker='+', linestyle='None')

    # hindsight_marker = Line2D([0], [0], label='Hindsight', color=CBLUE)
    # deterministic_marker = Line2D([0], [0], label='Deterministic', color=CGREEN)
    # hindsight_marker.set_linestyle('--')
    # deterministic_marker.set_linestyle('--')
    # handles.append(af_marker) 
    # handles.append(fm_marker) 
    # handles.append(fp_marker) 
    # handles.append(hindsight_marker) 
    # handles.append(deterministic_marker) 
    # fig.legend(handles=handles, bbox_to_anchor=(0.8, 0), ncol=5)
    if save:
        fig.savefig("./pdfs/2020_results_GA_HA.pdf")


#------------------------------------------------------------------------
#-----------------------------------------ALL THE OTHERS-------------------------
#------------------------------------------------------------------------


def plot_others(save=False):

    results_to_plot = [
        [results_g_all_PD, results_g_rf_PD, results_g_fm_PD],
        [results_h_all_PD, results_h_rf_PD, results_h_fm_PD],
        [results_g_all_PD_retrained, results_g_rf_PD_retrained, results_g_fm_PD_retrained],
        [results_h_all_PD_retrained, results_h_rf_PD_retrained, results_h_fm_PD_retrained]
    ]

    plot_titles = [
        "GAPD",
        "HAPD",
        "GAPDR",
        "HAPDR",
    ]

    fig, axs = plt.subplots(2,2, figsize=(10,10))
    xs = range(1,13)
    optimals = np.array([np.sum(result_hs['obj'])/(1e6) for _ in xs])
    deterministics = np.array([np.sum(result_det_strict['obj'])/(1e6) for _ in xs])
    for i, result_list in enumerate(results_to_plot):
        idx2 = int(np.floor((i/2)%3))
        idx1 = int(i%2)
        # print(f"idx1: {idx1}, idx2: {idx2}")
        axs[idx1, idx2].plot(xs, get_scaled_objs(result_list[0]), 'o', color=RED, label="AF")
        axs[idx1, idx2].plot(xs, get_scaled_objs(result_list[2]), '*', color=BLUE, label="FM")
        axs[idx1, idx2].plot(xs, get_scaled_objs(result_list[1]), '+', color=GREEN, alpha=0.8, label="RF")
        axs[idx1, idx2].plot(xs, optimals, '--', color=CBLUE, label="Hindsight")
        # axs[idx1, idx2].plot(xs, oracles[i]-0.03, '--', color="grey", label="Oracle")
        axs[idx1, idx2].plot(xs, deterministics, '--', color=CGREEN, label="Deterministic")
        axs[idx1, idx2].set_title(plot_titles[i], fontweight="bold")
        if idx2 > 0:
            axs[idx1, idx2].legend(loc="lower right")
        else:
            axs[idx1, idx2].legend()
        axs[idx1, idx2].set_xlabel("Months of training")
        axs[idx1, idx2].set_ylabel("Total revenue for test period [M€]")
        axs[idx1, idx2].set_ylim(1.6, 1.92)
        axs[idx1, idx2].set_xticks(xs)

    fig.tight_layout()
    if save:
        fig.savefig("./pdfs/2020_results_OTHERS.pdf")



# plot_ga_and_ha(True)
plot_others()

In [ ]:
#--------------------TESTING MISSING PRODUCTION

results_to_plot = [
    # [results_g_all, results_g_rf, results_g_fm],
    # [results_h_all, results_h_rf, results_h_fm],
    # [results_g_all_PD, results_g_rf_PD, results_g_fm_PD],
    # [results_h_all_PD, results_h_rf_PD, results_h_fm_PD],
    [results_g_all_PD_retrained, results_g_rf_PD_retrained, results_g_fm_PD_retrained],
    [results_h_all_PD_retrained, results_h_rf_PD_retrained, results_h_fm_PD_retrained]
]
plot_titles = [
    # "GA",
    # "HA",
    # "GAPD",
    # "HAPD",
    "GAPDR",
    "HAPDR",
]
feature_vector_names = [
    "AF",
    "FM",
    "RF"
]
for i, res_type in enumerate(results_to_plot):
    for j, res_list in enumerate(res_type):
        for k, res in enumerate(res_list):
            total_missing = np.sum(res['missing_production'])
            print(f"Model {plot_titles[i]}-{feature_vector_names[j]}-{k+1} has: {total_missing}")

In [ ]:
#-------------------REVENUE DISTRIBUTION----------------------
best_res = results_h_all_PD[0]
# oracle_res = results_h_all_pd_ORACLE[0]

strict_det = True
plot_penalties = True
plot_adjustments = False

# forward_opt = np.sum([result_hs['forward_bid'][i]*prices_F[t] for i, t in enumerate(range(idx_start, idx_end))])
# forward_det = np.sum([result_det_strict['forward_bid'][i]*prices_F[t] for i, t in enumerate(range(idx_start, idx_end))])
# forward_best = np.sum([best_res['forward_bid'][i]*prices_F[t] for i, t in enumerate(range(idx_start, idx_end))])
forward_opt = np.sum([result_hs['forward_bid'][i]*prices_F[t] for i, t in enumerate(range(idx_start, idx_end))  if (result_hs['forward_bid'][i] > 0)])
forward_det = np.sum([result_det_strict['forward_bid'][i]*prices_F[t] for i, t in enumerate(range(idx_start, idx_end))  if (result_det_strict['forward_bid'][i] > 0)])
forward_best = np.sum([best_res['forward_bid'][i]*prices_F[t] for i, t in enumerate(range(idx_start, idx_end))  if (best_res['forward_bid'][i] > 0)])

hprod_opt = np.sum([result_hs['h_prod'][i]*price_H for i, t in enumerate(range(idx_start, idx_end))])
hprod_det = np.sum([result_det_strict['h_prod'][i]*price_H for i, t in enumerate(range(idx_start, idx_end))])
hprod_best = np.sum([best_res['h_prod'][i]*price_H for i, t in enumerate(range(idx_start, idx_end))])


def dist_plot():

    if strict_det:
        det_res = result_det_strict
    else:
        det_res = result_det

    # xlabels = ["Hindsight", "Deterministic", "Learned", testname]
    xlabels = ["Hindsight", "Deterministic", "HAPD-AF-12"]
    # xlabels = ["Hindsight", "Deterministic", "95% quantile penalty", "Max penalty"]
    values = np.array([
        forward_opt,
        forward_det,
        forward_best
    ])

    hydrogens = np.array([
        hprod_opt,
        hprod_det,
        hprod_best
    ])

    values *= 1/1e6
    hydrogens *= 1/1e6

    xs = range(len(values))
    width = 0.35
    fig, ax = plt.subplots(figsize=(5,5))
    ax.grid(which='major', color='#DDDDDD', linewidth=0.8)
    ax.grid(which='minor', color='#EEEEEE', linestyle=':', linewidth=0.5)
    ax.minorticks_on()
    ax.set_axisbelow(True)
    ax.bar(xs, values, width, color=RED, label="Forward market revenue")
    ax.bar(xs, hydrogens, width, bottom=values, color=BLUE, label="Hydrogen production revenue")
    # ax.bar(xs, hydrogens, width, color=BLUE, label="Hydrogen production revenue")
    # if plot_penalties:
    #     ax.bar(xs, penalties, width, bottom=values, color=BLUE, label="Penalty Loss")
    # if plot_adjustments:
    #     ax.bar(xs, adjustments, width, bottom=values, color=BLUE, label="Adjustment Revenue")
    ax.set_xticklabels(xlabels, rotation=45)
    ax.set_xticks(xs)
    ax.set_ylabel("Total revenue for test period [M€]")
    ax.set_ylim(0, 3.2)
    fig.show()
    fig.tight_layout()
    ax.legend()
    print("\n")
    # print(f"Best is {(np.sum(best_res['obj'])-np.sum(other_res['obj']))/(np.sum(other_res['obj']))*100:.2f} % better than {testname}")
    # ax.set_title("2020 Results")
    return fig, ax

fig, ax = dist_plot()

# fig.savefig("./pdfs/rev_dist.pdf")

In [ ]:
#-------------------PENALTY, ADJUSTMENT, AND ORACLE COMPARISON PLOTS----------------------
ga_res = results_g_fm[0]
ga_oracle_res = results_g_fm_oracle[0]
hapd_res = results_h_all_PD[11]
hapd_oracle_res = results_h_all_pd_ORACLE[0]
ha_res = results_h_rf[0]
ha_oracle_res = results_h_rf_oracle[0]

best_res = results_h_all_PD[11]

strict_det = True
plot_penalties = False
plot_adjustments = False


def result_plot():

    if strict_det:
        det_res = result_det_strict
    else:
        det_res = result_det



    xlabels = ["Hindsight", "Deterministic", "GA-FM-1", "GA-FM-ORACLE"]
    # xlabels = ["Hindsight", "Deterministic", "HAPD-AF-12", "GA-AF-1"] 
    # xlabels = ["Hindsight", "Deterministic", "HAPD-AF-12", "HAPD-AF-ORACLE"]
    # xlabels = ["Hindsight", "Deterministic", "HAPD-AF-12", "HAPD-AF-12-OA"]

    values = np.array([
        np.sum(result_hs['obj']),
        np.sum(det_res['obj']),
        np.sum(ga_res['obj']),
        np.sum(ga_oracle_res['obj']),
        # obj(ha_oracle_res),
        # obj(best_res)
    ])

    # adjustments = np.array([
    #     0,
    #     np.sum(det_strict_up['obj'])-np.sum(det_res['obj']),
    #     np.sum(best_up['obj'])-np.sum(best_res['obj']),
    #     np.sum(result_hs_adj['obj'])-np.sum(best_res['obj']),
    # ])

    adjustments = np.array([
        0,
        np.sum(det_strict_up_dw['obj'])-np.sum(det_res['obj']),
        np.sum(best_up_dw['obj'])-np.sum(best_res['obj']),
        # obj(oracle_up_dw) - obj(hapd_oracle_res),
        np.sum(result_hs_adj['obj'])-np.sum(best_res['obj']),
    ])

    penalties = np.array([
        0,
        0,
        np.sum(best_res['missing_production'])*penalty,
        # obj(oracle_up_dw) - obj(hapd_oracle_res),
        np.sum(results_g_all[0]['missing_production'])*penalty,
    ])



    values *= 1/1e6
    adjustments *= 1/1e6
    penalties *= 1/1e6

    xs = range(len(values))
    width = 0.35
    fig, ax = plt.subplots(figsize=(5,5))
    ax.grid(which='major', color='#DDDDDD', linewidth=0.8)
    ax.grid(which='minor', color='#EEEEEE', linestyle=':', linewidth=0.5)
    ax.minorticks_on()
    ax.set_axisbelow(True)
    ax.bar(xs, values, width, color=RED, label="Final Revenue")
    if plot_penalties:
        ax.bar(xs, penalties, width, bottom=values, color=BLUE, label="Penalty Loss")
    if plot_adjustments:
        ax.bar(xs, adjustments, width, bottom=values, color=BLUE, label="Adjustment Revenue")
    ax.set_xticklabels(xlabels, rotation=45)
    ax.set_xticks(xs)
    ax.set_ylabel("Total revenue for test period [M€]")
    ax.set_ylim(0, 2.3)
    fig.show()
    fig.tight_layout()
    ax.legend()

    return fig, ax

fig, ax = result_plot()
# fig.savefig("./pdfs/eval_GA.pdf")
# fig.savefig("./pdfs/eval_HAPD.pdf")
# fig.savefig("./pdfs/adj_up_HAPD.pdf")
# fig.savefig("./pdfs/adj_dw_HAPD.pdf")
# fig.savefig("./pdfs/adj_opt.pdf")
# fig.savefig("./pdfs/adj_comparison.pdf")
# fig.savefig("./pdfs/eval_penalty.pdf")

In [ ]:
#------------------Comparison of adjustment results
def obj(res):
    return np.sum(res['obj'])

oracle_up, oracle_up_dw, oracle_pp = adjust(hapd_oracle_res)

def print_adj_res_strings(deto, detn, learno, learnn, oraco, oracn, opt_res):
    print(f"det increased with {(obj(detn) - obj(deto)) / obj(deto) * 100:.2f} % - {(obj(detn) - obj(deto)):.2f} €")
    print(f"learned increased with {(obj(learnn) - obj(learno)) / obj(learno) * 100:.2f} %  - {(obj(learnn) - obj(learno)):.2f} €")
    print(f"oracle increased with {(obj(oracn) - obj(oraco)) / obj(oraco) * 100:.2f} %  - {(obj(oracn) - obj(oraco)):.2f} €")
    print(f"learned is {(obj(learnn) - obj(detn)) / obj(detn) * 100:.2f} % better than det")
    print(f"oracle is {(obj(oracn) - obj(learnn)) / obj(learnn) * 100:.2f} % better than learned")
    print(f"hindsight is {(obj(opt_res) - obj(learnn)) / obj(learnn) * 100:.2f} % better than learned")

print_adj_res_strings(
    result_det_strict, 
    det_strict_up_dw, 
    best_res,
    best_up_dw,
    hapd_oracle_res,
    oracle_up_dw,
    result_hs
    )

In [ ]:
#---------------Print some high-level summaries for a model
def print_bal_info(res):
    b = np.sum(res['up'])
    s = np.sum(res['dw'])
    e = np.sum([res['dw'][i]*prices_S[t] for i, t in enumerate(range(idx_start, idx_end))])
    p = np.sum([res['up'][i]*prices_B[t] for i, t in enumerate(range(idx_start, idx_end))])
    f = np.sum([res['forward_bid'][i]*prices_F[t] for i, t in enumerate(range(idx_start, idx_end))])
    h = np.sum([res['h_prod'][i]*price_H for i, t in enumerate(range(idx_start, idx_end))])
    o = np.sum(res['obj'])
    # print("Total: ", s-b)
    # print("Surplus settlements: ", s, f" ({s / (s+b)*100:.2f}) %")
    # print("Deficit settlements: ", b, f" ({b / (s+b)*100:.2f}) %")
    print("Sum €: ", e+p)
    print("Total €: ", e-p)
    print("Surplus settlements €: ", e, f" ({e / (e+p)*100:.2f}) %")
    print("Deficit settlements €: ", p, f" ({p / (e+p)*100:.2f}) %")
    print("% of rev: ", (e+p)/(o-(e-p))*100)
    print("forward €: ", f)
    print("hydrogen €: ", h)
    print("Obj: ", o)
    print("Should-be-obj: ", f+h+e-p)

                

# print("\nOptimal")
# print_bal_info(result_hs)
print("\nDeterministic")
print_bal_info(result_det_strict)
print("\nLearned")
print_bal_info(best_res)

## Price analysis

##### Formatting price data

In [ ]:
years = [2019, 2020]
yearly_separated_prices = np.array([prices_F[:year], prices_F[year:]])
yearly_separated_prices_days = [[], []]

# Reformat prices to be an array of days
for idx_year, f_prices in enumerate(yearly_separated_prices):
    current_day = np.empty(24)
    for i, price in enumerate(f_prices):
        if i%24 == 0 and i != 0:
            yearly_separated_prices_days[idx_year].append(current_day)
            current_day = np.empty(24)
        current_day[i%24] = price
    yearly_separated_prices_days[idx_year].append(current_day)

# We now have yearly_separated_prices_days = [[365 days], [366 days]]

##### Plotting hourly mean prices

In [ ]:

df_train = pd.DataFrame(yearly_separated_prices_days[0])
df_test = pd.DataFrame(yearly_separated_prices_days[1])
# lgd = ax.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5,-0.1))

xs = range(24)
f = plt.figure()
plt.plot(xs, df_train.mean(), label="Training period", color=RED)
plt.plot(xs, df_test.mean(), label="Testing period", color=BLUE)
plt.xlabel("Hour of the day")
plt.ylabel("Mean price [€/MWh]")
lgd = plt.legend(loc=(1.05,0.8))
# plt.grid()
# plt.tight_layout()
plt.show()
f.savefig('mean_prices.pdf', bbox_extra_artists=(lgd,), bbox_inches='tight')


##### K-medoids clustering

In [ ]:
#-------------K-medoids clustering--------------------
# Find k medoid days in each year
k = 2

# medoids is an array of depth (2(5(24)))
initial_medoids = [price_year[:k] for price_year in yearly_separated_prices_days]
final_medoids = []

for idx_period, f_prices in enumerate(yearly_separated_prices_days):
    data = np.array(yearly_separated_prices_days[idx_period])
    KMobj = KMedoids(n_clusters=k).fit(data)
    # labels = KMobj.labels_
    final_medoids.append(KMobj.cluster_centers_)



In [ ]:
# ---------------Compare medoid days to each other
labels = ["Training period", "Testing period"]
dfs = [df_train, df_test]
## Plot for each year - good
fig, axs = plt.subplots(1, 2, figsize=(15,5))
for idx, (mlist, ax) in enumerate(zip(final_medoids, axs)):
    ax.set_ylim([0, 250])
    ax.set_xlim([0, 23])
    for p in yearly_separated_prices_days[idx]:
        ax.plot(range(24), p, "grey", alpha=0.2)
    for m in mlist:
        ax.plot(range(24), m)
    ax.plot(range(24), dfs[idx].mean(), 'k--', label="mean")
    ax.set_title(labels[idx])
    ax.set_ylabel("Forward price €/MWh")
    ax.set_xlabel("Hour of day")
    ax.legend()
plt.show()
# fig.savefig('k-medoids.pdf')


##### Wasserstein calculations

In [ ]:
from scipy.stats import wasserstein_distance

v_test = np.array([20-i for i in range(10)])
u_test = np.array([i for i in range(10)])

dist_matrix = np.empty((3,5,5))

all_medoids = final_medoids

mean_dists = {
    "2019-2020": 0,
    "2019-2021": 0,
    "2020-2021": 0,
    "2020-2021-red": 0,
    "2019-2021-red": 0,
    "2021-2021-red": 0,
    "2019-2019": 0,
    "2020-2020": 0,
    "2021-2021": 0,
    "2021-red-2021-red": 0
}


years_list = [[0, 1], [0, 2], [1, 2], [1, 3], [0, 3], [2, 3], [0, 0], [1, 1], [2, 2], [3,3]]

for i, key in enumerate(mean_dists.keys()):
    for from_idx in range(5):
        for to_idx in range(5):
            mean_dists[key] += wasserstein_distance(all_medoids[years_list[i][0]][from_idx], all_medoids[years_list[i][1]][to_idx])


mean_dists

# for y_idx in range(3):
#     for i in range(len(yearly_separated_prices_days[years_list[y_idx][0]])):
#         for j in range(len(yearly_separated_prices_days[years_list[y_idx][1]])):
            

## Detailed analysis of model output

In [ ]:
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.transforms as mtrans

offset = year
prices_F_test = prices_F[offset:]
prices_bal_test = [prices_B[offset+i] if round(prices_B[offset+i],0) < round(prices_F[offset+i],0) else prices_S[offset+i] for i in range(len(prices_S)-offset)]
prices_B_test = prices_B[offset:]
prices_S_test = prices_S[offset:]

def plot_res(res):
    fig, axs = plt.subplots(1,len(final_medoids[0]), figsize=(7*len(final_medoids[0]),5))
    handles = []
    fbid = mpatches.Patch(color=CGREEN, label='Forward bid [MWh]')
    hydro = mpatches.Patch(color=CBLUE, label='Hydrogen [MWh]')
    fprice = Line2D([0], [0], label='Forward price [€/MWh]', color=RED)
    fprod = Line2D([0], [0], label='Forecasted prod. [MWh]', color=BLUE)
    fprod.set_linestyle('--')
    handles.append(fbid) 
    handles.append(hydro) 
    handles.append(fprice) 
    handles.append(fprod) 
    for plotidx in range(len(final_medoids[0])):
        prods_learned = np.empty(24)
        forward_learned = np.empty(24)
        idx = KMobj.medoid_indices_[plotidx]*24
        for i, t in enumerate(range(idx, idx+24)):
            h = i%24
            prods_learned[h] = res['h_prod'][t]
            forward_learned[h] = res['forward_bid'][t]
        axs[plotidx].bar(range(24), prods_learned, label="Hydrogen", color=CBLUE)
        axs[plotidx].bar(range(24), forward_learned, label="Foward bid", color=CGREEN)
        # axs[plotidx].plot(range(24), realized[offset+idx:offset+idx+24], 'r--', label="Realized prod.")
        axs[plotidx].plot(range(24), features.loc[offset+idx:offset+idx+23, "production_FC"]*10, '--', label="Forecasted prod.")
        ax2 = axs[plotidx].twinx()
        ax2.plot(range(24), prices_F_test[idx:idx+24], color=RED, label="Forward price")
        # ax2.plot(range(24), prices_B_test[idx:idx+24], '*', color=RED, label="Deficit price")
        # ax2.plot(range(24), prices_S_test[idx:idx+24], '*', color=RED, label="Surplus price")
        ax2.plot(range(24), prices_bal_test[idx:idx+24], '*', color=RED, label="Balancing price")
        # fig.tight_layout()
        ax2.set_ylabel("Price [€/MWh]")
        axs[plotidx].set_ylabel("[MWh]")
        axs[plotidx].set_xlabel("Hour of the day")
        # fig.legend(bbox_to_anchor=(1.25, 0.85))
    fig.legend(handles=handles, bbox_to_anchor=(0.7, 0), ncol=4)
    fig.tight_layout()
    # Get the bounding boxes of the axes including text decorations
    r = fig.canvas.get_renderer()
    get_bbox = lambda ax: ax.get_tightbbox(r).transformed(fig.transFigure.inverted())
    bboxes = np.array(list(map(get_bbox, axs.flat)), mtrans.Bbox).reshape(axs.shape)
    # Draw a horizontal lines at those coordinates
    for y in [1/len(final_medoids[0])*(i+1) for i in range(len(final_medoids[0])-1)]:
        line = plt.Line2D([y,y],[0,1], transform=fig.transFigure, color="grey", alpha=0.6)
        fig.add_artist(line)

    fig.show()

plot_res(best_res)
plot_res(result_det_strict)
# plot_res(result_hs)


## Production data analysis

##### Formatting production data

In [ ]:
years = [2019, 2020, 2021]
#yearly_separated_prods = np.array([realized[:8760], realized[8760:8760+8784], realized[8760+8784:]])
yearly_separated_prods = np.array(realized[:8760] + realized[8760:8760+8784] + realized[8760+8784:])
yearly_separated_prods_days = [[], [], []]

# Reformat prods to be an array of days
for idx_year, f_prods in enumerate(yearly_separated_prods):
    current_day = np.empty(24)
    for i, prod in enumerate(f_prods):
        if i%24 == 0 and i != 0:
            yearly_separated_prods_days[idx_year].append(current_day)
            current_day = np.empty(24)
        current_day[i%24] = prod
    yearly_separated_prods_days[idx_year].append(current_day)

# We now have yearly_separated_prods_days = [[365 days], [366 days], [365 days]]

##### K-medoids 

In [ ]:
#-------------K-medoids clustering--------------------
# Find k medoid days in each year
k = 6

# medoids is an array of depth (3(5(24)))
initial_medoids_prods = [prod_year[:k] for prod_year in yearly_separated_prods_days]
final_medoids_prods = []

for idx_year, f_prices in enumerate(yearly_separated_prods_days):
    data = np.array(yearly_separated_prods_days[idx_year])
    KMobj = KMedoids(n_clusters=k).fit(data)
    # labels = KMobj.labels_
    final_medoids_prods.append(KMobj.cluster_centers_)

##### Plotting results

In [13]:
# ---------------Compare medoid days to each other
colors = ["GREEN", "BLUE", "RED"]
labels = ["2019", "2020", "2021"]

## Plot for each year - good
fig, axs = plt.subplots(1, 3, figsize=(15,5))
for idx, (mlist, ax) in enumerate(zip(final_medoids_prods, axs)):
    ax.set_ylim([0, 10])
    ax.set_xlim([0, 23])
    for p in yearly_separated_prods_days[idx]:
        ax.plot(range(24), p, "grey", alpha=0.2)
    for m in mlist:
        ax.plot(range(24), m)
    ax.set_title(f"Plot of {k} medoids for {labels[idx]}")
    ax.set_ylabel("Wind production [MW]")
    ax.set_xlabel("Hour of day")
plt.show()


In [ ]:

def plot_example_behavior():
    
    (prices_B,prices_S,prices_F,prices_bal,forecast,realized,price_H,max_elec,nominal_wind,penalty) = import_consts()

    year = 8760

    x_fm = pd.read_csv("./results/x_fm.csv")
    x_fm1 = x_fm["x_fm1"][-year:].to_numpy()
    x_fm2 = x_fm["x_fm2"][-year:].to_numpy()

    wind_last_year = realized[-year:]

    price_last_year = prices_F[-year:]
    hourly_prices = {}
    price_counter = 0
    hourly_prods = {}
    hourly_x1 = {}
    hourly_x2 = {}
    for i in range(24):
        hourly_prices[f"hour_{i}"] = 0
        hourly_prods[f"hour_{i}"] = 0
        hourly_x1[f"hour_{i}"] = 0
        hourly_x2[f"hour_{i}"] = 0
    for i in range(len(price_last_year)):
        # print("Tjek pris: ", price_last_year[i])
        if (price_last_year[i] > 35.2):
            price_counter += 1
            hourly_prices[f"hour_{i % 24}"] += price_last_year[i]
        hourly_prods[f"hour_{i % 24}"] += wind_last_year[i]
        hourly_x1[f"hour_{i%24}"] += x_fm1[i]
        hourly_x2[f"hour_{i%24}"] += x_fm2[i]

    hydrogen = np.zeros(24)
    adjustments = np.zeros(24)
    
    for i in range(24):
        hourly_prices[f"hour_{i}"] *= 1/(price_counter/24)
        hourly_prods[f"hour_{i}"] *= 1/(year/24)
        hourly_x1[f"hour_{i}"] *= 1/(year/24)
        hourly_x2[f"hour_{i}"] *= 1/(year/24)
            

    adjustments[2:4] = 10
    for i in range(5):
        hourly_prices[f"hour_{i}"] += 20

    for i in range(11, 15):
        
        hourly_prices[f"hour_{i}"] -= 20

    hydrogen[12:15] += 10
    hydrogen[-2:] += 10
    adjustments[-2:] = - 10
    prices = hourly_prices.values()
    x1s = hourly_x1.values()
    x2s = hourly_x2.values()

    prices_bal = np.array([hourly_prices[f"hour_{i}"] for i in range(24)])
    prices_bal[2:4] -= 50

    prices_bal[10:14] -= 4
    prices_bal[20] += 30
    prices_bal[23] += 40
    plt.bar(range(24), prices, label="Forward prices")
    # plt.bar(range(24), x1s)
    plt.plot(range(24), prices_bal, 'r', label="Balancing price")
    
    plt.bar(range(24), hydrogen, label="hydrogen production")
    plt.bar(range(24), adjustments, label="Adjustment")
    plt.legend()
    plt.title("Adjustment example")

    # plt.bar(range(len(price_last_two_months)), price_last_two_months)
    # print("Hydrogen prouction: ", np.sum(hydrogen))
plot_example_behavior()

In [ ]:

def plot_example_behavior_no_adj():
    
    (prices_B,prices_S,prices_F,prices_bal,forecast,realized,price_H,max_elec,nominal_wind,penalty) = import_consts()

    year = 8760

    x_fm = pd.read_csv("./results/x_fm.csv")
    x_fm1 = x_fm["x_fm1"][-year:].to_numpy()
    x_fm2 = x_fm["x_fm2"][-year:].to_numpy()

    wind_last_year = realized[-year:]

    price_last_year = prices_F[-year:]
    hourly_prices = {}
    price_counter = 0
    hourly_prods = {}
    hourly_x1 = {}
    hourly_x2 = {}
    for i in range(24):
        hourly_prices[f"hour_{i}"] = 0
        hourly_prods[f"hour_{i}"] = 0
        hourly_x1[f"hour_{i}"] = 0
        hourly_x2[f"hour_{i}"] = 0
    for i in range(len(price_last_year)):
        # print("Tjek pris: ", price_last_year[i])
        if (price_last_year[i] > 35.2):
            price_counter += 1
            hourly_prices[f"hour_{i % 24}"] += price_last_year[i]
        hourly_prods[f"hour_{i % 24}"] += wind_last_year[i]
        hourly_x1[f"hour_{i%24}"] += x_fm1[i]
        hourly_x2[f"hour_{i%24}"] += x_fm2[i]

    hydrogen = np.zeros(24)
    adjustments = np.zeros(24)
    
    for i in range(24):
        hourly_prices[f"hour_{i}"] *= 1/(price_counter/24)
        hourly_prods[f"hour_{i}"] *= 1/(year/24)
        hourly_x1[f"hour_{i}"] *= 1/(year/24)
        hourly_x2[f"hour_{i}"] *= 1/(year/24)
            

    adjustments[2:4] = 10
    for i in range(5):
        hourly_prices[f"hour_{i}"] += 20

    for i in range(11, 15):
        
        hourly_prices[f"hour_{i}"] -= 20

    hydrogen[12:15] += 10
    hydrogen[-2:] += 10
    adjustments[-2:] = - 10
    prices = hourly_prices.values()
    x1s = hourly_x1.values()
    x2s = hourly_x2.values()

    prices_bal = np.array([hourly_prices[f"hour_{i}"] for i in range(24)])
    prices_bal[2:4] -= 50

    prices_bal[10:14] += 30
    prices_bal[20] += 30
    prices_bal[23] += 40
    plt.bar(range(24), prices, label="Forward prices")
    # plt.bar(range(24), x1s)
    # plt.plot(range(24), prices_bal, 'r', label="Balancing price")
    
    plt.bar(range(24), hydrogen, label="hydrogen production")
    # plt.bar(range(24), adjustments, label="Adjustment")
    plt.legend()
    plt.title("Deterministic example")

    # plt.bar(range(len(price_last_two_months)), price_last_two_months)
    # print("Hydrogen prouction: ", np.sum(hydrogen))
plot_example_behavior_no_adj()

In [ ]:

def plot_deterministic_behavior():
    
    (prices_B,prices_S,prices_F,prices_bal,forecast,realized,price_H,max_elec,nominal_wind,penalty) = import_consts()

    year = 8760

    x_fm = pd.read_csv("./results/x_fm.csv")
    x_fm1 = x_fm["x_fm1"][-year:].to_numpy()
    x_fm2 = x_fm["x_fm2"][-year:].to_numpy()

    wind_last_year = realized[-year:]

    price_last_year = prices_F[-year:]
    hourly_prices = {}
    price_counter = 0
    hourly_prods = {}
    hourly_x1 = {}
    hourly_x2 = {}
    for i in range(24):
        hourly_prices[f"hour_{i}"] = 0
        hourly_prods[f"hour_{i}"] = 0
        hourly_x1[f"hour_{i}"] = 0
        hourly_x2[f"hour_{i}"] = 0
    for i in range(len(price_last_year)):
        # print("Tjek pris: ", price_last_year[i])
        if (price_last_year[i] > 35.2):
            price_counter += 1
            hourly_prices[f"hour_{i % 24}"] += price_last_year[i]
        hourly_prods[f"hour_{i % 24}"] += wind_last_year[i]
        hourly_x1[f"hour_{i%24}"] += x_fm1[i]
        hourly_x2[f"hour_{i%24}"] += x_fm2[i]

    hydrogen = np.zeros(24)
    
    for i in range(24):
        hourly_prices[f"hour_{i}"] *= 1/(price_counter/24)
        hourly_prods[f"hour_{i}"] *= 1/(year/24)
        hourly_x1[f"hour_{i}"] *= 1/(year/24)
        hourly_x2[f"hour_{i}"] *= 1/(year/24)
        if i < 5:
            hydrogen[i] += 10


    prices = hourly_prices.values()
    x1s = hourly_x1.values()
    x2s = hourly_x2.values()


    plt.bar(range(24), prices, label="Forward prices")
    # plt.bar(range(24), x1s)
    # plt.bar(range(24), forward, label="forward bid")
    plt.bar(range(24), hydrogen, label="hydrogen production")
    plt.legend()
    plt.title("Deterministic behavior")

    # plt.bar(range(len(price_last_two_months)), price_last_two_months)
    # print("Hydrogen prouction: ", np.sum(hydrogen))


plot_deterministic_behavior()

In [ ]:

def plot_mean_behavior(df_qs):
    
    (prices_B,prices_S,prices_F,prices_bal,forecast,realized,price_H,max_elec,nominal_wind,penalty) = import_consts()

    year = 8760

    x_fm = pd.read_csv("./results/x_fm.csv")
    x_fm1 = x_fm["x_fm1"][-year:].to_numpy()
    x_fm2 = x_fm["x_fm2"][-year:].to_numpy()

    wind_last_year = realized[-year:]

    price_last_year = prices_F[-year:]
    hourly_prices = {}
    price_counter = 0
    hourly_prods = {}
    hourly_x1 = {}
    hourly_x2 = {}
    for i in range(24):
        hourly_prices[f"hour_{i}"] = 0
        hourly_prods[f"hour_{i}"] = 0
        hourly_x1[f"hour_{i}"] = 0
        hourly_x2[f"hour_{i}"] = 0
    for i in range(len(price_last_year)):
        # print("Tjek pris: ", price_last_year[i])
        if (price_last_year[i] > 35.2):
            price_counter += 1
            hourly_prices[f"hour_{i % 24}"] += price_last_year[i]
        hourly_prods[f"hour_{i % 24}"] += wind_last_year[i]
        hourly_x1[f"hour_{i%24}"] += x_fm1[i]
        hourly_x2[f"hour_{i%24}"] += x_fm2[i]

    for i in range(24):
        hourly_prices[f"hour_{i}"] *= 1/(price_counter/24)
        hourly_prods[f"hour_{i}"] *= 1/(year/24)
        hourly_x1[f"hour_{i}"] *= 1/(year/24)
        hourly_x2[f"hour_{i}"] *= 1/(year/24)


    prices = hourly_prices.values()
    prods = hourly_prods.values()
    x1s = hourly_x1.values()
    x2s = hourly_x2.values()
    hydrogen = []
    forward = []
    for i, prod, price in zip(range(24), x1s, x2s):
        hydrogen.append(np.maximum((df_qs["qH1"][i]*prod + df_qs["qH2"][i]*price + df_qs["qH3"][i]),0))
        if (price > 35.2):
            forward.append(np.maximum((df_qs["qF1"][i]*prod + df_qs["qF2"][i]*price + df_qs["qF3"][i]),0))
        else:
            forward.append(0)

        # print(f'For hour {i}, FC={prod}, FP={price}, FB={(df_qs["qF1"][i]*prod + df_qs["qF2"][i]*price + df_qs["qF3"][i])}, H={(df_qs["qH1"][i]*prod + df_qs["qH2"][i]*price + df_qs["qH3"][i])}')
        

    balancing_price = np.array([hourly_prices[f"hour_{i}"] for i in range(24)])
    balancing_price[:5] += 35
    balancing_price[11:15] -= 20
    plt.bar(range(24), prices, label="Forward prices")
    # plt.bar(range(24), x1s)
    # plt.bar(range(24), forward, label="forward bid")
    plt.plot(range(24), balancing_price, 'r', label="balancing price")
    plt.bar(range(24), hydrogen, label="hydrogen production")
    plt.legend()
    plt.title("Learned behavior")

    # plt.bar(range(len(price_last_two_months)), price_last_two_months)
    print("Hydrogen prouction: ", np.sum(hydrogen))


df_initial = pd.read_csv("./results/RES_05.2_mo_24_5.csv")
plot_mean_behavior(df_initial)